In [1]:
import sys
import os

# Ruta al directorio raíz del proyecto (ajústala si es necesario)
ruta_raiz = os.path.abspath(r"C:\Users\Germán Llorente\Desktop\germiprogramer\TFG-Agente-de-Scouting")

if ruta_raiz not in sys.path:
    sys.path.append(ruta_raiz)

In [2]:
import importlib
import funciones_analisis.estadisticas

importlib.reload(funciones_analisis.estadisticas)
importlib.reload(funciones_analisis.generales)

from funciones_analisis.estadisticas import *
from funciones_analisis.generales import *
from statsbombpy import sb
import pandas as pd
from mplsoccer import VerticalPitch, Pitch
from highlight_text import ax_text, fig_text
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.patheffects as path_effects
import seaborn as sns
import glob

In [3]:
# Cargar los datasets
jugadores_laliga = pd.read_csv('datos/datos_jugadores_v3/jugadores_laliga.csv', encoding="utf-8", skipinitialspace=True)
jugadores_premier = pd.read_csv('datos/datos_jugadores_v3/jugadores_premier.csv')
jugadores_seriea = pd.read_csv('datos/datos_jugadores_v3/jugadores_seriea.csv')
jugadores_bundesliga = pd.read_csv('datos/datos_jugadores_v3/jugadores_bundesliga.csv')

In [4]:
for i, df in enumerate([jugadores_laliga, jugadores_premier, jugadores_seriea, jugadores_bundesliga], start=1):
    print(f"\nDataset {i} columnas:")
    print(list(df.columns))



Dataset 1 columnas:
['player_id', 'player_name', 'team', 'team_id', 'posiciones', 'partidos_jugados', 'overall', 'potential', 'value_eur', 'wage_eur', 'age', 'dob', 'height_cm', 'weight_kg', 'club_jersey_number', 'club_loaned_from', 'club_contract_valid_until', 'nationality_id', 'nationality_name', 'preferred_foot', 'release_clause_eur', 'competition', 'tackle_success_rate', 'tackles_successful', 'interception_success_rate', 'interceptions', 'clearances', 'blocks', 'head_clearances', 'head_clearances_won', 'head_clearance_success_rate', 'headed_shots_total', 'headed_shots_after_duel', 'headed_shot_duel_rate', 'total_passes', 'completed_passes', 'incomplete_passes', 'passes_out', 'offside_passes', 'failed_passes', 'pass_completion_rate', 'avg_pass_length', 'ground_passes', 'low_passes', 'high_passes', 'ground_pass_percentage', 'low_pass_percentage', 'high_pass_percentage', 'crosses_total', 'crosses_completed', 'cutbacks_total', 'cutbacks_completed', 'switches_total', 'switches_complete

In [5]:
# Unir todos los datasets (uno debajo del otro)
jugadores_total = pd.concat([jugadores_laliga, jugadores_premier, jugadores_seriea, jugadores_bundesliga])
jugadores_total

,player_id,player_name,team,team_id,posiciones,partidos_jugados,overall,potential,value_eur,wage_eur,...,key_passes_per90,chances_created_per90,goals_conceded_per90,xg_total_per90,goals_minus_xg_per90,PSxG_per90,psxg_minus_goals_conceded_per90,yellow_cards_per90,red_cards_per90,posicion_mas_jugada
0,3023,Yuri Berchiche Izeta,Real Sociedad,210,['Left Back'],21,71,76,1800000.0,20000.0,...,0.598772,0.783009,0.000000,0.017631,-0.017503,0.000000,0.000000,0.460594,0.0,Left Back
1,3063,Danilo Luiz da Silva,Real Madrid,220,"['Right Back', 'Left Back', 'Right Wing']",24,80,86,16500000.0,90000.0,...,0.505618,0.716292,0.000000,0.040390,0.043820,0.000000,0.000000,0.252809,0.0,Right Back
2,3084,Christian Atsu Twasam,Málaga,223,"['Left Midfield', 'Right Midfield', 'Left Cent...",11,63,69,375000.0,4000.0,...,1.956522,1.956522,0.000000,0.410496,0.241304,0.000000,0.000000,0.000000,0.0,Right Midfield
3,3130,Gaël Kakuta,Sevilla,213,"['Left Midfield', 'Right Wing']",2,76,78,6500000.0,60000.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.967742,0.0,Left Midfield
4,3160,Rene Krhin,Granada,1049,"['Right Defensive Midfield', 'Left Defensive M...",23,75,80,4500000.0,35000.0,...,0.240642,0.240642,0.000000,0.031147,-0.031283,0.000000,0.000000,0.180481,0.0,Right Defensive Midfield
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,400621,Tohouri Zahoui Constant Djakpa,Eintracht Frankfurt,184,"['Left Back', 'Left Wing Back', 'Left Midfield...",11,75,75,3100000.0,40000.0,...,0.851351,0.851351,0.000000,0.019767,-0.019459,0.000000,0.000000,0.243243,0.0,Left Back
424,400622,Joel Gerezgiher,Eintracht Frankfurt,184,['Left Midfield'],3,60,72,300000.0,2000.0,...,0.978261,0.978261,0.000000,0.304519,-0.303261,0.000000,0.000000,0.978261,0.0,Left Midfield
425,400627,Dario Kresic,Bayer Leverkusen,904,['Goalkeeper'],1,68,68,675000.0,10000.0,...,0.000000,0.000000,1.914894,0.000000,0.000000,1.780851,-0.134043,0.000000,0.0,Goalkeeper
426,400654,Roel Brouwers,Borussia Mönchengladbach,185,"['Right Center Back', 'Left Center Back', 'Rig...",8,76,76,2300000.0,50000.0,...,0.000000,0.000000,0.000000,0.025924,-0.025806,0.000000,0.000000,0.000000,0.0,Right Center Back


In [6]:
import pandas as pd

# Lista de (player_id, team) válidos
ids_y_equipos_validos = [
    (3083, "Tottenham Hotspur"),
    (3089, "Manchester City"),
    (3302, "Watford"),
    (5474, "Inter Milan"),
    (5687, "Eibar"),
    (6593, "Real Betis"),
    (6743, "Rayo Vallecano"),
    (6950, "Swansea City"),
    (7141, "Levante UD"),
    (7170, "Carpi"),
    (7788, "Torino"),
    (7852, "Genoa"),
    (8217, "Juventus"),
    (9461, "Norwich City"),
    (10881, "Sevilla"),
    (17520, "VfB Stuttgart"),
    (18404, "Rayo Vallecano"),
    (23488, "Watford"),
    (26211, "Levante UD"),
    (27341, "Celta Vigo"),
    (27853, "Fiorentina"),
]

# Convertir a DataFrame
df_validos = pd.DataFrame(ids_y_equipos_validos, columns=['player_id', 'team'])

# Filtrar duplicados por player_id
duplicados = jugadores_total[jugadores_total.duplicated(subset='player_id', keep=False)]

# Mantener solo los duplicados correctos (player_id + team que coincidan con la lista)
duplicados_validos = duplicados.merge(df_validos, on=['player_id', 'team'])

# Mantener los jugadores únicos que no están duplicados
no_duplicados = jugadores_total[~jugadores_total['player_id'].isin(duplicados['player_id'])]

# Unir los válidos con los no duplicados
jugadores_total = pd.concat([no_duplicados, duplicados_validos], ignore_index=True)

# Ver resultado



In [7]:
# Filtrar jugadores que están cedidos (club_loaned_from no es NaN)
cedidos = jugadores_total[jugadores_total['club_loaned_from'].notna()]

# Mostrar algunas columnas relevantes
print(cedidos[['player_id', 'player_name', 'team', 'club_loaned_from']])


      player_id                    player_name                    team  \
10         3379                Alphonse Areola              Villarreal   
23         4418           Matheus Dória Macedo                 Granada   
65         5678   Jefferson Andrés Lerma Solís              Levante UD   
68         5719        Marco Asensio Willemsen                Espanyol   
70         5724                    Fayçal Fajr  RC Deportivo La Coruña   
...         ...                            ...                     ...   
1738      10902                Philipp Hosiner                 FC Köln   
1782      16502            László Kleinheisler           Werder Bremen   
1889       6743  Manuel Rolando Iturra Urrutia          Rayo Vallecano   
1890       7141                 Giuseppe Rossi              Levante UD   
1899       9461                     Timm Klose            Norwich City   

            club_loaned_from  
10       Paris Saint-Germain  
23    Olympique de Marseille  
65            Atlé

In [8]:
jugadores_total["team"].unique()

array(['Real Sociedad', 'Real Madrid', 'Málaga', 'Sevilla', 'Granada',
       'Valencia', 'Villarreal', 'Barcelona', 'Real Betis', 'Las Palmas',
       'Athletic Club', 'Rayo Vallecano', 'Espanyol', 'Atlético Madrid',
       'Celta Vigo', 'RC Deportivo La Coruña', 'Levante UD', 'Eibar',
       'Sporting Gijón', 'Getafe', 'Chelsea', 'West Bromwich Albion',
       'Manchester United', 'Tottenham Hotspur', 'Aston Villa',
       'Newcastle United', 'AFC Bournemouth', 'Manchester City',
       'West Ham United', 'Crystal Palace', 'Southampton', 'Sunderland',
       'Everton', 'Swansea City', 'Watford', 'Stoke City',
       'Leicester City', 'Norwich City', 'Liverpool', 'Arsenal', 'Torino',
       'Lazio', 'Fiorentina', 'AS Roma', 'AC Milan', 'Empoli', 'Genoa',
       'Inter Milan', 'Juventus', 'Atalanta', 'Udinese', 'Napoli',
       'Sassuolo', 'Hellas Verona', 'Palermo', 'Sampdoria', 'Bologna',
       'Chievo', 'Carpi', 'Frosinone', 'Ingolstadt', 'Hannover 96',
       'Schalke 04', 'Bayer 

In [9]:
jugadores_total["club_loaned_from"].unique()

array([nan, 'Paris Saint-Germain', 'Olympique de Marseille',
       'Atlético Huila', 'Real Madrid CF', 'Elche CF', 'AC Milan',
       'SC Braga', 'Atlético de Madrid', 'Rubin Kazan', 'Granada CF',
       'Derby County', 'Valencia CF', 'FC Porto', 'Watford', 'SL Benfica',
       'Roma', 'FC Barcelona', 'Inter', 'Liverpool', 'Galatasaray SK',
       'SV Werder Bremen', 'Villarreal CF', "CD O'Higgins", 'Sporting CP',
       'Celtic', 'Udinese Calcio', 'Cruz Azul', 'Norwich City',
       'Sunderland', 'Swansea City', 'AS Monaco', 'Tottenham Hotspur',
       'Chelsea', 'Stade Rennais FC', 'Fulham', 'Arsenal',
       'West Ham United', 'VfB Stuttgart', 'Zenit St. Petersburg',
       'Borussia Dortmund', 'U.S. Sassuolo Calcio', 'Palermo', 'Pescara',
       'Napoli', 'U.C. Sampdoria', 'Genoa', 'Juventus', 'Cagliari',
       'Grasshopper Club Zürich', 'FC Sion', 'Fenerbahçe SK',
       'Hellas Verona', 'Standard de Liège', 'Club Brugge KV',
       'Sevilla FC', 'Bologna', 'FC Bayern München',


In [10]:
import numpy as np

# Función para truncar a 2 decimales (sin redondear) y formatear como string con dos decimales
def truncar_y_formatear(x):
    return f"{np.floor(x * 100) / 100:.2f}"

# Detectar columnas numéricas con decimales (float)
columnas_decimales = jugadores_total.select_dtypes(include='float').columns

# Aplicar el truncado y formateo
for col in columnas_decimales:
    jugadores_total[col] = jugadores_total[col].apply(truncar_y_formatear)

# Ver resultado en las columnas modificadas
print(jugadores_total[columnas_decimales].head())


     value_eur  wage_eur club_jersey_number club_contract_valid_until  \
0   1800000.00  20000.00              19.00                   2016.00   
1  16500000.00  90000.00              23.00                   2021.00   
2    375000.00   4000.00              27.00                   2016.00   
3   6500000.00  60000.00              12.00                   2019.00   
4   4500000.00  35000.00              21.00                   2019.00   

  release_clause_eur tackle_success_rate interception_success_rate  \
0                nan               56.86                     62.75   
1                nan               64.18                     61.11   
2                nan               66.67                     66.67   
3                nan              100.00                      0.00   
4                nan               64.70                     71.05   

  head_clearance_success_rate headed_shot_duel_rate pass_completion_rate  ...  \
0                       51.43                  0.00        

In [11]:
#jugadores_total.to_csv("datos/datos_jugadores_v3/jugadores_total.csv")

In [12]:
df_ratings = pd.read_csv("datos/datos_jugadores_v3/jugadores_mundial.csv")

In [13]:
df_ratings

,Unnamed: 0,competition,date,match,team,pos,pos_role,player_name,match_id,minutes_played,...,key_passes_per90,chances_created_per90,goals_conceded_per90,xg_total_per90,goals_minus_xg_per90,PSxG_per90,psxg_minus_goals_conceded_per90,yellow_cards_per90,red_cards_per90,posicion_mas_jugada
0,4886,World Cup 2018,2018-07-03,"Colombia - England, 1 - 1",Colombia,DF,DC,Davinson Sánchez Mina,7585,120,...,0.000000,0.000000,0.0,0.138741,-0.135000,0.0,0.0,0.000000,0.0,Left Center Back
1,4890,World Cup 2018,2018-07-03,"Colombia - England, 1 - 1",Colombia,DF,DC,Yerry Fernando Mina González,7585,120,...,0.000000,0.000000,0.0,0.127085,0.622500,0.0,0.0,0.000000,0.0,Right Center Back
2,4893,World Cup 2018,2018-07-03,"Colombia - England, 1 - 1",Colombia,DF,DL,Johan Andrés Mojica Palacio,7585,120,...,1.500000,1.500000,0.0,0.004635,-0.007500,0.0,0.0,0.000000,0.0,Left Back
3,4897,World Cup 2018,2018-07-03,"Colombia - England, 1 - 1",Colombia,DF,DR,Santiago Arias Naranjo,7585,116,...,0.775862,0.775862,0.0,0.000000,0.000000,0.0,0.0,0.775862,0.0,Right Back
4,4901,World Cup 2018,2018-07-03,"Colombia - England, 1 - 1",Colombia,FW,FW,Juan Guillermo Cuadrado Bello,7585,120,...,2.250000,3.000000,0.0,0.076088,-0.075000,0.0,0.0,0.750000,0.0,Right Wing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1701,3024,World Cup 2018,2018-06-25,"Saudi Arabia - Egypt, 2 - 1",Saudi Arabia,MF,MC,Hussain Al Mogahwi,7559,90,...,2.000000,2.000000,0.0,0.169422,-0.170000,0.0,0.0,0.000000,0.0,Right Center Midfield
1702,3026,World Cup 2018,2018-06-25,"Saudi Arabia - Egypt, 2 - 1",Saudi Arabia,MF,MC,Salman Mohammed Al Faraj,7559,90,...,5.000000,5.000000,0.0,0.969338,0.030000,0.0,0.0,0.000000,0.0,Left Center Midfield
1703,3029,World Cup 2018,2018-06-25,"Saudi Arabia - Egypt, 2 - 1",Saudi Arabia,MF,ML,Salem Mohammed Al Dawsari,7559,90,...,2.000000,2.000000,0.0,0.196278,0.800000,0.0,0.0,0.000000,0.0,Left Midfield
1704,3032,World Cup 2018,2018-06-25,"Saudi Arabia - Egypt, 2 - 1",Saudi Arabia,MF,MR,Hattan Babhir,7559,65,...,0.000000,0.000000,0.0,0.116950,-0.110769,0.0,0.0,0.000000,0.0,Right Midfield


In [14]:
jugadores_total.rename(columns={'posicion_mas_jugada': 'main_position'}, inplace=True)
df_ratings.rename(columns={'posicion_mas_jugada': 'main_position'}, inplace=True)

In [15]:
df_ratings[df_ratings["main_position"] == "Goalkeeper"]


,Unnamed: 0,competition,date,match,team,pos,pos_role,player_name,match_id,minutes_played,...,key_passes_per90,chances_created_per90,goals_conceded_per90,xg_total_per90,goals_minus_xg_per90,PSxG_per90,psxg_minus_goals_conceded_per90,yellow_cards_per90,red_cards_per90,main_position
6,4909,World Cup 2018,2018-07-03,"Colombia - England, 1 - 1",Colombia,GK,GK,David Ospina Ramírez,7585,90,...,0.0,0.0,1.000000,0.0,0.0,0.793316,-0.210000,0.0,0.0,Goalkeeper
20,4967,World Cup 2018,2018-07-03,"Colombia - England, 1 - 1",England,GK,GK,Jordan Pickford,7585,90,...,0.0,0.0,1.000000,0.0,0.0,0.227255,-0.770000,0.0,0.0,Goalkeeper
34,3903,World Cup 2018,2018-06-28,"England - Belgium, 0 - 1",Belgium,GK,GK,Thibaut Courtois,7570,90,...,0.0,0.0,0.000000,0.0,0.0,0.460367,0.460000,0.0,0.0,Goalkeeper
48,3963,World Cup 2018,2018-06-28,"England - Belgium, 0 - 1",England,GK,GK,Jordan Pickford,7570,90,...,0.0,0.0,1.000000,0.0,0.0,0.152233,-0.850000,0.0,0.0,Goalkeeper
62,5049,World Cup 2018,2018-07-03,"Sweden - Switzerland, 1 - 0",Sweden,GK,GK,Robin Olsen,7586,90,...,0.0,0.0,0.000000,0.0,0.0,0.087946,0.090000,0.0,0.0,Goalkeeper
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649,3387,World Cup 2018,2018-06-26,"Denmark - France, 0 - 0",France,MF,DMC,Steve Mandanda,7563,90,...,0.0,0.0,0.000000,0.0,0.0,0.008513,0.010000,0.0,0.0,Goalkeeper
1658,2743,World Cup 2018,2018-06-24,"Japan - Senegal, 2 - 2",Japan,GK,GK,Eiji Kawashima,7556,90,...,0.0,0.0,2.000000,0.0,0.0,0.923394,-1.080000,0.0,0.0,Goalkeeper
1673,2790,World Cup 2018,2018-06-24,"Japan - Senegal, 2 - 2",Senegal,MF,MC,Khadim N''Diaye,7556,81,...,0.0,0.0,2.222222,0.0,0.0,0.909205,-1.311111,0.0,0.0,Goalkeeper
1685,2984,World Cup 2018,2018-06-25,"Saudi Arabia - Egypt, 2 - 1",Egypt,GK,GK,Essam Kamal Tawfik El Hadary,7559,90,...,0.0,0.0,2.000000,0.0,0.0,1.898497,-0.100000,0.0,0.0,Goalkeeper


In [16]:
jugadores_total["minutes_played"] = pd.to_numeric(jugadores_total["minutes_played"], errors="coerce")
jugadores_total["aerial_dominance_index_per90"] = pd.to_numeric(jugadores_total["aerial_dominance_index"], errors="coerce")

df_ratings["minutes_played"] = pd.to_numeric(df_ratings["minutes_played"], errors="coerce")
df_ratings["aerial_dominance_index_per90"] = pd.to_numeric(df_ratings["aerial_dominance_index"], errors="coerce")




C:\Users\Germán Llorente\AppData\Local\Temp\ipykernel_48748\900931858.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  jugadores_total["aerial_dominance_index_per90"] = pd.to_numeric(jugadores_total["aerial_dominance_index"], errors="coerce")


In [17]:
jugadores_total

,player_id,player_name,team,team_id,posiciones,partidos_jugados,overall,potential,value_eur,wage_eur,...,chances_created_per90,goals_conceded_per90,xg_total_per90,goals_minus_xg_per90,PSxG_per90,psxg_minus_goals_conceded_per90,yellow_cards_per90,red_cards_per90,main_position,aerial_dominance_index_per90
0,3023,Yuri Berchiche Izeta,Real Sociedad,210,['Left Back'],21,71,76,1800000.00,20000.00,...,0.78,0.00,0.01,-0.02,0.00,0.00,0.46,0.00,Left Back,0.0
1,3063,Danilo Luiz da Silva,Real Madrid,220,"['Right Back', 'Left Back', 'Right Wing']",24,80,86,16500000.00,90000.00,...,0.71,0.00,0.04,0.04,0.00,0.00,0.25,0.00,Right Back,0.0
2,3084,Christian Atsu Twasam,Málaga,223,"['Left Midfield', 'Right Midfield', 'Left Cent...",11,63,69,375000.00,4000.00,...,1.95,0.00,0.41,0.24,0.00,0.00,0.00,0.00,Right Midfield,0.0
3,3130,Gaël Kakuta,Sevilla,213,"['Left Midfield', 'Right Wing']",2,76,78,6500000.00,60000.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.96,0.00,Left Midfield,0.0
4,3160,Rene Krhin,Granada,1049,"['Right Defensive Midfield', 'Left Defensive M...",23,75,80,4500000.00,35000.00,...,0.24,0.00,0.03,-0.04,0.00,0.00,0.18,0.00,Right Defensive Midfield,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1903,7788,Ciro Immobile,Torino,241,"['Left Center Forward', 'Right Center Forward'...",14,78,85,12000000.00,50000.00,...,1.84,0.00,0.50,-0.07,0.00,0.00,0.17,0.00,Left Center Forward,0.0
1904,7852,Tim Matavž,Genoa,233,"['Right Defensive Midfield', 'Right Wing', 'Ce...",7,81,88,20500000.00,70000.00,...,0.77,0.00,0.14,-0.15,0.00,0.00,0.00,0.00,Left Center Forward,0.0
1905,8217,Kingsley Coman,Juventus,224,['Left Center Forward'],1,75,75,2600000.00,50000.00,...,1.87,0.00,0.15,-0.15,0.00,0.00,0.00,0.00,Left Center Forward,0.0
1906,27853,Mauro Matías Zárate,Fiorentina,239,"['Center Forward', 'Right Center Forward', 'Le...",15,79,80,11000000.00,90000.00,...,1.61,0.00,0.43,0.04,0.00,0.00,0.32,0.16,Left Wing,0.0


In [18]:
# Variables que quieres normalizar
metricas_minmax = [
    'tackle_success_rate', 'interception_success_rate', 'head_clearance_success_rate',
    'headed_shot_duel_rate', 'pass_completion_rate', 'avg_pass_length', 'progressive_passes_accuracy',
    'passes_final_third_accuracy', 'passes_to_box_accuracy', 'progressive_carries_rate',
    'through_balls_completed', 'avg_carry_distance', 'duel_success_rate', 'dribble_success_rate',
    'minutes_played', 'shot_accuracy', 'avg_xg', 'conversion_rate_inside_box', 'save_percentage',
    'tackles_successful_per90', 'interceptions_per90', 'clearances_per90', 'blocks_per90',
    'head_clearances_per90', 'head_clearances_won_per90', 'times_dribbled_past_per90',
    'ball_recoveries_per90', 'offensive_recoveries_per90', 'pressures_per90', 'counterpress_per90',
    'total_passes_per90', 'completed_passes_per90', 'incomplete_passes_per90', 'passes_out_per90',
    'offside_passes_per90', 'failed_passes_per90', 'ground_passes_per90', 'low_passes_per90',
    'high_passes_per90', 'deflected_passes_per90', 'crosses_total_per90', 'crosses_completed_per90',
    'cutbacks_total_per90', 'cutbacks_completed_per90', 'switches_total_per90', 'switches_completed_per90',
    'through_balls_total_per90', 'through_balls_completed_per90', 'passes_own_half_per90',
    'passes_opposition_half_per90', 'progressive_passes_per90', 'progressive_passes_completed_per90',
    'passes_final_third_per90', 'passes_to_box_per90', 'carries_per90', 'progressive_carries_per90',
    'dribbles_completed_per90', 'duels_total_per90', 'duels_won_per90', 'fouls_won_per90',
    'fouls_committed_per90', 'penalties_won_per90', 'penalties_conceded_per90', 'penalties_saved_per90',
    'saves_per90', 'keeper_sweeper_per90', 'shots_total_per90', 'non_blocked_shots_per90',
    'shots_on_target_per90', 'shots_off_target_per90', 'blocked_shots_per90', 'shots_inside_box_per90',
    'free_kick_goals_per90', 'goals_scored_per90', 'penalty_goals_per90', 'goal_assists_per90',
    'key_passes_per90', 'chances_created_per90', 'goals_conceded_per90', 'xg_total_per90',
    'goals_minus_xg_per90', 'PSxG_per90', 'psxg_minus_goals_conceded_per90', 'yellow_cards_per90',
    'red_cards_per90', 'penalty_save_percentage', 'aerial_dominance_index_per90'
]


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler


# Diccionario de variables por posición
estadisticas_posiciones = {
    "goalkeeper": ["saves_per90", "save_percentage", "psxg_minus_goals_conceded_per90", 
                   "keeper_sweeper_per90", "penalty_save_percentage", "aerial_dominance_index_per90"],
    "center_back": ["tackle_success_rate", "tackles_successful_per90", "interceptions_per90", "interception_success_rate", "clearances_per90", 
                    "blocks_per90", "duels_won_per90", "times_dribbled_past_per90", "progressive_passes_per90", "passes_completed_per90", "duel_success_rate", "goals_scored_per90"],
    "side_back": ["tackle_success_rate", "tackles_successful_per90", "interceptions_per90", "interception_success_rate", "pressures_per90", 
                  "progressive_carries_per90", "crosses_completed_per90", "duels_won_per90", "fouls_committed_per90", "goals_scored_per90", "goal_assists_per90"],
    "defensive_midfield": ["interceptions_per90", "ball_recoveries_per90", "pressures_per90", 
                           "progressive_passes_per90", "duels_won_per90", "fouls_committed_per90", "goals_scored_per90", "goal_assists_per90", "duel_success_rate"],
    "center_midfield": ["progressive_passes_per90", "chances_created_per90", "dribbles_completed_per90", 
                        "duels_won_per90", "pressures_per90", "goals_scored_per90", "goal_assists_per90", "dribble_success_rate", "duel_success_rate"],
    "offensive_midfield": ["goal_assists_per90", "key_passes_per90", "progressive_passes_per90", 
                           "shots_total_per90", "xg_total_per90", "dribbles_completed_per90", "goals_scored_per90", "dribble_success_rate"],
    "winger": ["dribbles_completed_per90", "dribble_success_rate", "crosses_completed_per90", "key_passes_per90", 
               "goal_assists_per90", "progressive_carries_per90", "goals_scored_per90", "goals_minus_xg_per90"],
    "striker": ["xg_total_per90", "shot_accuracy", 
                "goal_assists_per90", "penalties_won_per90", "key_passes_per90"]
}

# Diccionario de etiquetas por posición
etiquetas_posiciones = {
    "goalkeeper": ["Goalkeeper"],
    "center_back": ['Center Back', 'Right Center Back', 'Left Center Back'],
    "side_back": ['Left Back', 'Right Back', 'Right Wing Back', 'Left Wing Back'],
    "defensive_midfield": ['Right Defensive Midfield', 'Center Defensive Midfield', 'Left Defensive Midfield'],
    "center_midfield": ['Right Center Midfield', 'Center Midfield', 'Left Center Midfield'],
    "offensive_midfield": ['Left Attacking Midfield', 'Center Attacking Midfield', 'Right Attacking Midfield'],
    "winger": ['Left Center Forward', 'Right Center Forward', 'Left Wing', 'Left Midfield', 'Right Wing', 'Right Midfield'],
    "striker": ['Striker', 'Secondary Striker']
}

# Variables que quieres conservar tal cual
variables_extra = ['main_position', 'player_id', 'player_name', 'team', 'team_id']


In [ ]:
import pandas as pd

def normalizar_metricas_percentiles(
    df, 
    metricas, 
    variables_extra, 
    posiciones_filtrar=None
):
    """
    Normaliza métricas usando percentiles (0-100).
    
    Parámetros:
    - df: DataFrame original
    - metricas: lista de columnas a normalizar
    - variables_extra: lista de columnas a mantener sin cambios
    - posiciones_filtrar: None (no filtra) o lista de valores de 'main_position' a filtrar
    
    Devuelve:
    - DataFrame normalizado por percentiles
    """
    
    # Si se especifican posiciones, filtramos
    if posiciones_filtrar is not None:
        df = df[df['main_position'].isin(posiciones_filtrar)].copy()
    
    # Comprobamos qué métricas existen realmente en el DataFrame
    columnas_existentes = [col for col in metricas if col in df.columns]
    
    # Crear nuevo DataFrame con métricas y variables extra
    df_percentiles = df[variables_extra + columnas_existentes].copy()
    
    # Convertimos a numérico las columnas de métricas
    for col in columnas_existentes:
        df_percentiles[col] = pd.to_numeric(df_percentiles[col], errors='coerce')
    
    # Aplicar percentiles (0-100)
    for col in columnas_existentes:
        df_percentiles[col] = df_percentiles[col].rank(pct=True) * 100

    return df_percentiles

df_porteros_percentiles = normalizar_metricas_percentiles(jugadores_total, estadisticas_posiciones["goalkeeper"], variables_extra, etiquetas_posiciones["goalkeeper"])
df_laterales_percentiles = normalizar_metricas_percentiles(jugadores_total, estadisticas_posiciones["side_back"], variables_extra, etiquetas_posiciones["side_back"])
df_centrales_percentiles = normalizar_metricas_percentiles(jugadores_total, estadisticas_posiciones["center_back"], variables_extra, etiquetas_posiciones["center_back"])
df_centrocampista_defensivo_percentiles = normalizar_metricas_percentiles(jugadores_total, estadisticas_posiciones["defensive_midfield"], variables_extra, etiquetas_posiciones["defensive_midfield"])
df_centrocampista_percentiles = normalizar_metricas_percentiles(jugadores_total, estadisticas_posiciones["center_midfield"], variables_extra, etiquetas_posiciones["center_midfield"])
df_centrocampista_ofensivo_percentiles = normalizar_metricas_percentiles(jugadores_total, estadisticas_posiciones["offensive_midfield"], variables_extra, etiquetas_posiciones["offensive_midfield"])
df_extremo_percentiles = normalizar_metricas_percentiles(jugadores_total, estadisticas_posiciones["winger"], variables_extra, etiquetas_posiciones["winger"])
df_delantero_percentiles = normalizar_metricas_percentiles(jugadores_total, estadisticas_posiciones["striker"], variables_extra, etiquetas_posiciones["striker"])


In [39]:
df_porteros_percentiles.head(5)

,player_id,player_name,team,team_id,posiciones,partidos_jugados,overall,potential,value_eur,wage_eur,...,chances_created_per90,goals_conceded_per90,xg_total_per90,goals_minus_xg_per90,PSxG_per90,psxg_minus_goals_conceded_per90,yellow_cards_per90,red_cards_per90,main_position,aerial_dominance_index_per90
6,3240,Mathew Ryan,Valencia,207,['Goalkeeper'],8,78,83,9500000.00,80000.00,...,0.00,1.08,0.00,0.00,0.58,68.181818,0.00,0.00,Goalkeeper,52.121212
10,3379,Alphonse Areola,Villarreal,222,['Goalkeeper'],32,76,85,7000000.00,40000.00,...,0.00,0.75,0.00,0.00,0.45,85.757576,0.03,0.00,Goalkeeper,93.939394
26,4691,Claudio Andrés Bravo Muñoz,Barcelona,217,['Goalkeeper'],32,83,83,17500000.00,130000.00,...,0.00,0.60,0.00,0.00,0.41,91.515152,0.00,0.00,Goalkeeper,45.151515
58,5577,Francisco Guillermo Ochoa Magaña,Málaga,223,['Goalkeeper'],11,80,80,12000000.00,60000.00,...,0.00,0.67,0.00,0.00,0.48,91.515152,0.00,0.00,Goalkeeper,40.909091
61,5597,Keylor Navas Gamboa,Real Madrid,220,['Goalkeeper'],34,79,80,10000000.00,110000.00,...,0.02,0.79,0.00,0.00,0.56,88.484848,0.00,0.00,Goalkeeper,77.575758


In [ ]:
def normalizar_metricas_minmax(df, metricas, variables_extra, posiciones_filtrar=None):
    """
    Normaliza métricas usando min-max scaling (0-100).

    Parámetros:
    - df: DataFrame original
    - metricas: lista de columnas a normalizar
    - variables_extra: lista de columnas a mantener sin cambios
    - posiciones_filtrar: None o lista de posiciones de 'main_position' a filtrar

    Devuelve:
    - DataFrame normalizado con valores entre 0 y 100
    """
    # Filtrado opcional
    if posiciones_filtrar is not None:
        df = df[df['main_position'].isin(posiciones_filtrar)].copy()
    
    # Seleccionar columnas existentes
    columnas_existentes = [col for col in metricas if col in df.columns]
    
    # Crear DataFrame nuevo
    df_minmax = df[variables_extra + columnas_existentes].copy()
    
    # Convertir métricas a numérico
    for col in columnas_existentes:
        df_minmax[col] = pd.to_numeric(df_minmax[col], errors='coerce')
    
    # Aplicar normalización min-max
    for col in columnas_existentes:
        col_values = df_minmax[col]
        min_val = col_values.min()
        max_val = col_values.max()
        if pd.notnull(min_val) and pd.notnull(max_val) and max_val != min_val:
            df_minmax[col] = (col_values - min_val) / (max_val - min_val) * 100
        else:
            df_minmax[col] = 0  # Para columnas constantes o vacías

    return df_minmax


In [ ]:
def calcular_rating(df, tipo='minmax'):
    """
    Calcula el rating ponderado de los jugadores según su posición.
    
    Parámetros:
    - df: DataFrame normalizado (min-max o percentiles)
    - tipo: 'minmax' o 'percentiles' (solo para referencia, no afecta al cálculo ahora)
    
    Devuelve:
    - DataFrame con una columna nueva 'rating'
    """
    # Definir ponderaciones por posición
    ponderaciones = {
        'Goalkeeper': {
            'saves_per90': 0.25,
            'save_percentage': 0.20,
            'psxg_minus_goals_conceded_per90': 0.20,
            'keeper_sweeper_per90': 0.15,
            'penalty_save_percentage': 0.10,
            'aerial_dominance_index_per90': 0.10,
            'pass_completion_rate': 0.05,
            
        },
        'Center Back': {
            'tackle_success_rate': 0.15,
            'interceptions_per90': 0.15,
            'clearances_per90': 0.15,
            'blocks_per90': 0.15,
            'head_clearances_per90': 0.10,
            'headed_shot_duel_rate': 0.10,
            'completed_passes_per90': 0.10,
            'progressive_passes_per90': 0.05,
            'times_dribbled_past_per90': 0.05  # ¡Importante invertirlo después!
        },
        'Side Back': {
            'tackle_success_rate': 0.15,
            'interceptions_per90': 0.10,
            'pressures_per90': 0.10,
            'progressive_carries_per90': 0.15,
            'crosses_completed_per90': 0.15,
            'dribbles_completed_per90': 0.10,
            'key_passes_per90': 0.10,
            'progressive_passes_per90': 0.10,
            'fouls_committed_per90': 0.05  # ¡Invertir!
        },
        'Defensive Midfield': {
            'interceptions_per90': 0.20,
            'ball_recoveries_per90': 0.20,
            'pressures_per90': 0.15,
            'tackle_success_rate': 0.15,
            'progressive_passes_per90': 0.15,
            'completed_passes_per90': 0.10,
            'fouls_committed_per90': 0.05  # ¡Invertir!
        },
        'Center Midfield': {
            'progressive_passes_per90': 0.20,
            'chances_created_per90': 0.20,
            'pressures_per90': 0.15,
            'dribbles_completed_per90': 0.15,
            'key_passes_per90': 0.10,
            'tackles_successful_per90': 0.10,
            'progressive_carries_per90': 0.10
        },
        'Offensive Midfield': {
            'goal_assists_per90': 0.20,
            'key_passes_per90': 0.20,
            'chances_created_per90': 0.20,
            'dribbles_completed_per90': 0.15,
            'xg_total_per90': 0.15,
            'progressive_passes_per90': 0.10
        },
        'Winger': {
            'dribbles_completed_per90': 0.20,
            'progressive_carries_per90': 0.15,
            'crosses_completed_per90': 0.15,
            'key_passes_per90': 0.15,
            'xg_total_per90': 0.15,
            'goal_assists_per90': 0.10,
            'shots_on_target_per90': 0.10
        },
        'Striker': {
            'xg_total_per90': 0.25,
            'goals_scored_per90': 0.25,
            'shot_accuracy': 0.20,
            'shots_inside_box_per90': 0.10,
            'penalty_goals_per90': 0.05,
            'key_passes_per90': 0.05,
            'fouls_won_per90': 0.10
        }
    }
    
    df = df.copy()
    ratings = []
    
    for idx, row in df.iterrows():
        pos = row['main_position']
        rating = 0
        if pos in ponderaciones:
            for stat, weight in ponderaciones[pos].items():
                if stat in row:
                    valor = row[stat]
                    # Invertimos las métricas negativas (times_dribbled_past_per90, fouls_committed_per90)
                    if stat in ['times_dribbled_past_per90', 'fouls_committed_per90']:
                        valor = 100 - valor
                    rating += valor * weight
        ratings.append(rating)
    
    df['rating'] = ratings
    return df
